In [2]:
pip install content 

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.



  Created wheel for content: filename=content-0.0.1-py3-none-any.whl size=1401 sha256=13f42e7b936b6746c35ec1a38056e964ed2dad784b7b44b978b7de761fb46311
  Stored in directory: c:\users\prith\appdata\local\pip\cache\wheels\c6\bb\0d\cece9b6510ed46c9c0a8877fb773e2b08b82f8538a8a93456b
Successfully built content


In [3]:
import pandas as pd
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from concurrent.futures import ProcessPoolExecutor
import json 
import uuid
from tqdm import tqdm_notebook as tqdm 

In [15]:
data = pd.read_csv('dataset/flipkart_com-ecommerce_sample.csv')

In [17]:
data = pd.read_csv('dataset/flipkart_com-ecommerce_sample.csv')

def get_first_image_url(df):
    try:
        images_urls = df['image'].replace('["','').replace('"]','').split('", "')
        first_image = images_urls[0]

        return first_image
    except:
        return None
    
image_url = data.apply(lambda x:get_first_image_url(x),axis=1)
data['image_url'] = image_url

In [18]:
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,image_url
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",http://img5a.flixcart.com/image/short/u/4/a/al...
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",http://img6a.flixcart.com/image/sofa-bed/j/f/y...
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",http://img5a.flixcart.com/image/shoe/7/z/z/red...
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",http://img5a.flixcart.com/image/short/6/2/h/al...
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",http://img5a.flixcart.com/image/pet-shampoo/r/...


In [29]:
import get_images # get image function is defined in seperate python file 

In [30]:
with ProcessPoolExecutor(max_workers = None) as executor:
    results = list(tqdm(executor.map(get_images.get_image,
                                     data.uniq_id.values,
                                     data.product_name.values,
                                     data.product_category_tree.values,
                                     data.image_url.values),
                total = len(data)))

<ipython-input-30-6b2b05d25dae>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  results = list(tqdm(executor.map(get_images.get_image,


  0%|          | 0/20000 [00:00<?, ?it/s]

In [32]:
# Create Metadata csv
metadata = pd.concat(results,axis=0)

metadata.to_csv('Dataset/metadata.csv',index=False)

In [33]:
def str_test(df,size):
  string = str(df.category).replace('["','').replace('>> ','').replace('"]','')
  string = string[0:size]
  return string

In [34]:
a = pd.DataFrame()
a['category'] = metadata['category']
a['category_2'] = a.apply(lambda x: str_test(x, 30), axis=1)

In [35]:
len(a.category.unique()),len(a.category_2.unique())

(5940, 664)

In [36]:
a.category_2.unique()

array(["Clothing Women's Clothing Ling", 'Furniture Living Room Furnitur',
       "Footwear Women's Footwear Ball", 'Pet Supplies Grooming Skin & C',
       'Eternal Gandhi Super Series Cr', "Clothing Women's Clothing Spor",
       'Pens & Stationery School Suppl', "Clothing Kids' Clothing Boys W",
       "Footwear Women's Footwear Spor", 'Pet Supplies Toys Comfort Toys',
       'Sports & Fitness Other Sports ', 'Beauty and Personal Care Makeu',
       "Clothing Women's Clothing Fusi", "Clothing Men's Clothing Cargos",
       'Bengal Blooms Rose Artificial ', 'Pens & Stationery Diaries & No',
       "Clothing Women's Clothing Form", 'Pens & Stationery Office Suppl',
       'Bags, Wallets & Belts Bags Pou', "Footwear Women's Footwear Casu",
       'Home Decor & Festive Needs Sho', 'Automotive Accessories & Spare',
       'Tools & Hardware Tools Hardwar', "Vishudh Printed Women's Straig",
       "Vishudh Printed Women's Anarka", 'BuildTrack PIR Wireless Motion',
       'Skayvon SUMMERSIB